In [1]:
import sys
import importlib
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from google.colab import files

In [2]:
import sys
sys.path.append('/content')

import myNN
import importlib; importlib.reload(myNN)

print(dir(myNN))  # checking stuff


['ZScoreLayer', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'h5py', 'intList', 'layerNum', 'loadWeights', 'load_model', 'model', 'os', 'tf']


In [3]:
import h5py

# Open the h5 file to inspect its structure
with h5py.File('/content/myNN/weights.h5', 'r') as f:
    # List all groups in the file
    print("Keys in weights.h5:", list(f.keys()))

    # Optionally, explore the first layer or a specific layer
    layer_name = list(f.keys())[0]  # Adjust if needed
    print(f"Structure of layer {layer_name}:")
    print(f[layer_name].attrs)


Keys in weights.h5: ['fc_1_', 'fc_2_', 'fc_3_', 'fc_4_', 'fc_5_', 'fc_6_', 'input_']
Structure of layer fc_1_:
<Attributes of HDF5 object at 139107669924480>


In [4]:
with h5py.File('/content/myNN/weights.h5', 'r') as f:
    # List all groups (layers) in the file
    for group_name in f:
        group = f[group_name]
        print(f"Layer: {group_name}")
        print("    Number of variables:", group.attrs.get('NumVars'))
        print("    Weights:", list(group.keys()))


Layer: fc_1_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: fc_2_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: fc_3_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: fc_4_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: fc_5_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: fc_6_
    Number of variables: [2.]
    Weights: ['bias', 'kernel']
Layer: input_
    Number of variables: [2.]
    Weights: ['mean', 'stdev']


In [5]:
model = myNN.load_model()

print("Mean shape:", model.get_layer("input_").mean.shape)
print("Mean (first 5):", model.get_layer("input_").mean.numpy()[:5])

model.export('/content/my_saved_NN')

Mean shape: (1024,)
Mean (first 5): [ 71.138466  75.62285   79.72336   87.17977  107.68489 ]


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_unnormalized']
Received: inputs=Tensor(shape=(None, 1024))
  warnings.warn(msg)


Saved artifact at '/content/my_saved_NN'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1024), dtype=tf.float32, name='input_unnormalized')
Output Type:
  TensorSpec(shape=(None, 9), dtype=tf.float32, name=None)
Captures:
  139107317313424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126380112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126383568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126384144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126386064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126384336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126383760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126386640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126383376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139107126387216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13910712638

In [6]:
!zip -r my_saved_NN.zip my_saved_NN

files.download("my_saved_NN.zip")

  adding: my_saved_NN/ (stored 0%)
  adding: my_saved_NN/saved_model.pb (deflated 86%)
  adding: my_saved_NN/variables/ (stored 0%)
  adding: my_saved_NN/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: my_saved_NN/variables/variables.index (deflated 66%)
  adding: my_saved_NN/assets/ (stored 0%)
  adding: my_saved_NN/fingerprint.pb (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# Print mean and std from ZScoreLayer
zlayer = None
for layer in model.layers:
    if isinstance(layer, myNN.ZScoreLayer):
        zlayer = layer
        break

if zlayer:
    print("Found ZScoreLayer")
    print("Mean:", zlayer.mean.numpy())
    print("Std:", zlayer.std.numpy())
else:
    print("ZScoreLayer not found in model.")

Found ZScoreLayer
Mean: [ 71.138466  75.62285   79.72336  ...  95.46716  106.117645  84.66297 ]
Std: [24.997116 27.195084 26.85353  ... 35.568363 41.665295 31.921804]


In [9]:
# === Load and Preprocess Test Data ===
test_df = pd.read_csv('/content/test_7_9label.csv')  # ← Update path if needed

# Separate features and labels
X_test = test_df[[f'var{i}' for i in range(1, 1025)]].values
y_test = test_df['posture'].values

# Encode labels to integers (e.g., 'correct' → 1, 'incorrect' → 0, 'not_sitting' → 2)
label_map = {
    'cross_legged': 0,
    'leaning_forward': 1,
    'leaning_left': 2,
    'leaning_left_lc': 3,
    'leaning_right': 4,
    'leaning_right_lc': 5,
    'not_sitting': 6,
    'slouching': 7,
    'straight': 8
}
y_test_encoded = np.array([label_map[label] for label in y_test])

# === Run Inference ===
y_pred_probs = model.predict({'input_unnormalized': X_test})
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # Assumes model output is softmax

# === Evaluate ===
print("Accuracy:", accuracy_score(y_test_encoded, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_encoded, y_pred_classes))
print("Classification Report:\n", classification_report(
    y_test_encoded,
    y_pred_classes,
    target_names=['cross_legged', 'leaning_forward', 'leaning_left', 'leaning_left_lc', 'leaning_right', 'leaning_right_lc', 'not_sitting', 'slouching', 'straight']
))


258/258 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Accuracy: 0.6404903507707246
Confusion Matrix:
 [[661   0  12   0   0   0  15 128   0]
 [  0 927   0   0   7   0   0 168   0]
 [ 51   0 474 259   0   2   0   2 130]
 [  0   0 313 584   2  48   2   1  83]
 [  0   1  46   0 467 374   1   0 142]
 [  0   0   2 114 103 760   0   2   0]
 [  0   0   0   0   0   0 419   0   0]
 [  0 313  19   0   0   0   1 334 164]
 [ 41   1   3 121   3 140   3 145 651]]
Classification Report:
                   precision    recall  f1-score   support

    cross_legged       0.88      0.81      0.84       816
 leaning_forward       0.75      0.84      0.79      1102
    leaning_left       0.55      0.52      0.53       918
 leaning_left_lc       0.54      0.57      0.55      1033
   leaning_right       0.80      0.45      0.58      1031
leaning_right_lc       0.57      0.77      0.66       981
     not_sitting       0.95      1.00      0.97       419
       slouching       0.43      0.40      0.41       831
        stra

In [10]:
import tensorflow as tf

# Path to your SavedModel directory
saved_model_dir = "/content/my_saved_NN"

# Converter
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# (Optional) Enable optimizations like quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save it to a file
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


In [11]:
# ================== TESTING TFLITE MODEL ======================

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])


Input shape: [   1 1024]
Output shape: [1 9]


In [12]:
correct = 0
total = len(X_test)

for i in range(total):
    input_data = X_test[i:i+1].astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    pred = np.argmax(output)

    if pred == y_test_encoded[i]:
        correct += 1

print("TFLite accuracy:", correct / total)


TFLite accuracy: 0.6401262289112757
